In [76]:
from astropy.io import fits
from matplotlib import pyplot as plt
% matplotlib inline
from astropy.stats import sigma_clip
from astropy.stats import biweight_midvariance as bimv
from astropy.stats import biweight_location as bil
import numpy as np
import numpy.ma as ma
import scipy as sp
from scipy import signal
import operator
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import splprep, splev
from collections import Counter

In [91]:
def medpoly(file, kappa=3, iterations=5, sigma=2, poly=4):
    im = fits.getdata(file)
    sc = sigma_clip(im, sigma=kappa, iters=iterations)
    medfilt=sp.signal.medfilt(sc)
    rows = ma.mean(medfilt,axis=1)
    y = list(range(len(rows)))
    values=[]
    
    for j in range(poly,poly+10):
        medpoly = np.poly1d(np.polyfit(y,rows,j))
        flattened = rows - medpoly(y)

        for i in range(len(rows)):
            mean = ma.mean(flattened)
            sig = ma.std(flattened)
            out = ma.where(flattened[i]>(mean+sigma*sig))[0]
            if out.size:
                values.append(i)
                
    print(Counter(values))

In [93]:
medpoly('Fepdsses20180219T072418.2_084_sci_L.fits')
medpoly('Fepdsses20180219T070215.4_084_sci_L.fits')
medpoly('Fepdsses20180219T071318.8_084_sci_L.fits')

Counter({149: 10, 63: 10, 182: 9, 150: 9, 62: 9, 151: 8, 64: 7, 65: 7, 61: 4, 1: 2, 113: 2, 2: 1, 183: 1, 221: 1, 220: 1})
Counter({150: 10, 151: 10, 149: 10, 61: 5, 62: 4, 1: 2, 5: 2, 2: 1, 4: 1, 221: 1})
Counter({149: 10, 150: 10, 151: 10, 113: 8, 61: 5, 62: 5, 112: 4, 1: 2, 2: 2, 63: 2, 182: 1, 221: 1})


In [105]:
def polymedpoly(file, kappa=3, iterations=5, sigma=2, poly=4, length=10):
    
    values=[]
    
    for k in file:
        im = fits.getdata(k)
        sc = sigma_clip(im, sigma=kappa, iters=iterations)
        medfilt=sp.signal.medfilt(sc)
        rows = ma.mean(medfilt,axis=1)
        y = list(range(len(rows)))

        for j in range(poly,poly+length):
            medpoly = np.poly1d(np.polyfit(y,rows,j))
            flattened = rows - medpoly(y)

            for i in range(len(rows)):
                mean = ma.mean(flattened)
                sig = ma.std(flattened)
                out = ma.where(flattened[i]>(mean+sigma*sig))[0]
                if out.size:
                    values.append(i)

    print(Counter(values))

In [121]:
files=['Fepdsses20180219T072418.2_084_sci_L.fits',
       'Fepdsses20180219T070215.4_084_sci_L.fits',
       'Fepdsses20180219T071318.8_084_sci_L.fits']
polymedpoly(files)

Counter({149: 30, 150: 29, 151: 28, 62: 18, 61: 14, 63: 12, 182: 10, 113: 10, 64: 7, 65: 7, 1: 6, 2: 4, 112: 4, 221: 3, 5: 2, 4: 1, 183: 1, 220: 1})
